In [1]:
from helper_random_forest import *
import warnings
warnings.filterwarnings("ignore")

h2o.init()
# h2o.no_progress()

_, train, test, _ = load_data_rf('lc_db_processed.csv', 'binary')

# Per tuning parameters
# _, db, _, _ = load_data_rf('lc_db_processed.csv', 'binary')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Simone\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Simone\AppData\Local\Temp\tmp1maovgki
  JVM stdout: C:\Users\Simone\AppData\Local\Temp\tmp1maovgki\h2o_Simone_started_from_python.out
  JVM stderr: C:\Users\Simone\AppData\Local\Temp\tmp1maovgki\h2o_Simone_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.1
H2O_cluster_version_age:,10 days
H2O_cluster_name:,H2O_from_python_Simone_zjb4e9
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.643 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [2]:
cv_DistributedRandomForest_H2O(db)

Mon Apr  5 14:30:44 2021
Epoch 1:  Mon Apr  5 14:30:44 2021
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


H2OServerError: HTTP 500 Server Error:
Server error java.lang.RuntimeException:
  Error: java.lang.StackOverflowError
  Request: None


In [4]:
target = 'conc1_mean'
predictors = train.columns.to_list()
predictors.remove(target)

train_h2o = h2o.H2OFrame(train)
test_h2o = h2o.H2OFrame(test)

for i in train.columns:
    if train[i].dtypes == 'object':
        train_h2o[i] = train_h2o[i].asfactor()
        test_h2o[i] = test_h2o[i].asfactor()

train_h2o['conc1_mean'] = train_h2o['conc1_mean'].asfactor()
test_h2o['conc1_mean'] = test_h2o['conc1_mean'].asfactor()

drf = H2ORandomForestEstimator(seed = 123, categorical_encoding = 'onehotexplicit', binomial_double_trees = True)
drf.train(x = predictors, y = target, training_frame = train_h2o)
        
y_pred = drf.predict(test_h2o).as_data_frame()['predict']
y_test = test_h2o['conc1_mean'].as_data_frame()

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print(accuracy_score(y_pred, y_test))
print(recall_score(y_pred, y_test))
print(f1_score(y_pred, y_test))

print(tn/(tn+fp))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
drf Model Build progress: |███████████████████████████████████████████████| 100%


H2OServerError: HTTP 500 Server Error:
Server error java.lang.RuntimeException:
  Error: java.lang.StackOverflowError
  Request: None
